In [2]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


Using TensorFlow backend.


In [3]:
classifier = Sequential()
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Add a second convolutional layer, and you don't need an input shape parameter because Keras already knows from the previous set
classifier.add(Convolution2D(32, 3, 3, activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(output_dim= 128, activation='relu'))
classifier.add(Dense(output_dim= 1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
cwd = '/home/devinlax/jupyter/Artificial_Neural_Networks/convolution/dataset/'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [1]:
training_set = train_datagen.flow_from_directory(
        cwd + 'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


NameError: name 'train_datagen' is not defined

In [11]:

test_set = test_datagen.flow_from_directory(
        cwd + 'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 2000 images belonging to 2 classes.


In [12]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=1,
        validation_data=test_set,
        validation_steps=2000)



Epoch 1/1
8000/8000 [==============================] - 1130s 141ms/step - loss: 0.3883 - accuracy: 0.8164 - val_loss: 0.4996 - val_accuracy: 0.7955


In [13]:
classifier.save_weights('dog_cat_cnn.h5') # Save the model for later use

# from keras.models import Sequential
# from keras.layers import Conv2d, MaxPooling2D, Dense, Flatten

### build the model
classifier = Sequential()
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Aier.add(Flatten())
classifier.add(Dense(output_dim= 128, activation='relu'))
classifier.add(Dense(output_dim= 1, activation='sigmoid'))
# classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#])

### model meclassifier.load_weights('name_of_model.h5')

In [27]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
img = load_img('test_image.jpg')
img_array = img_to_array(img)
img_array_expanded = np.expand_dims(img_array, axis=0)
classifier.predict(np.asarray(np.reshape(img_array_expanded, (64, 64), order='C' )))

ValueError: cannot reshape array of size 151686 into shape (64,64)